<a href="https://colab.research.google.com/github/mohithxoxo/Auto_ML/blob/master/TPOT_AutoML_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Audit Dataset from UCI**
small size dataset

In [75]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
import time
_start_time = time.time()
def tic():
    global _start_time 
    _start_time = time.time()

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00475/audit_data.zip
!unzip /content/audit_data.zip

--2020-09-21 05:22:56--  https://archive.ics.uci.edu/ml/machine-learning-databases/00475/audit_data.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28447 (28K) [application/x-httpd-php]
Saving to: ‘audit_data.zip’

audit_data.zip      100%[===================>]  27.78K  --.-KB/s    in 0.03s   

2020-09-21 05:22:56 (916 KB/s) - ‘audit_data.zip’ saved [28447/28447]

Archive:  /content/audit_data.zip
  inflating: audit_data/audit_risk.csv  
  inflating: audit_data/trial.csv    


In [34]:
data = pd.read_csv("/content/audit_data/audit_risk.csv")
data = data.dropna()
data = data.drop(columns=['LOCATION_ID'])

## using only three columns as predictors/independent variables
X, y = data.iloc[:,1:4], data.iloc[:, -1]
print(X.shape, y.shape) 
X = np.array(X)
y = np.array(y)

(775, 3) (775,)


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

training_features, testing_features, training_target, testing_target = \
            train_test_split(X,y, random_state=1)

#Randomly selected algorithm with default parameters 
tic()
exported_pipeline = RandomForestClassifier()
exported_pipeline.fit(training_features, training_target)
tac()
results = exported_pipeline.predict(testing_features)
print(accuracy_score(testing_target,results))
print(classification_report(testing_target,results))

Time passed: 0hour:0min:0sec
0.7525773195876289
              precision    recall  f1-score   support

           0       0.76      0.85      0.80       116
           1       0.73      0.60      0.66        78

    accuracy                           0.75       194
   macro avg       0.75      0.73      0.73       194
weighted avg       0.75      0.75      0.75       194



In [ ]:
!pip install tpot
from sklearn.model_selection import RepeatedKFold
from tpot import TPOTClassifier

In [37]:
#### Using TPOT to identify perfect algorithm + parameter

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
model = TPOTClassifier(generations=5, population_size=50,scoring='accuracy', cv=cv,verbosity=2, random_state=1, n_jobs=-1)

tic()
# perform the search
model.fit(X, y)
# export the best model
model.export('tpot_audit_best_model.py')
tac()


Generation 1 - Current best internal CV score: 0.8163725163725163
Generation 2 - Current best internal CV score: 0.8163780663780662
Generation 3 - Current best internal CV score: 0.8163780663780662
Generation 4 - Current best internal CV score: 0.8163780663780662
Generation 5 - Current best internal CV score: 0.8167721167721168
Best pipeline: XGBClassifier(Normalizer(input_matrix, norm=max), learning_rate=0.01, max_depth=4, min_child_weight=18, n_estimators=100, nthread=1, subsample=0.5)
Time passed: 0hour:9min:17sec


In [46]:
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from tpot.export_utils import set_param_recursive
from xgboost import XGBClassifier

# Parameters identified by TPOT 
training_features, testing_features, training_target, testing_target = \
            train_test_split(X, y, random_state=1)

# Average CV score on the training set was: 0.8167721167721168
exported_pipeline = make_pipeline(
    Normalizer(norm="max"),
    XGBClassifier(learning_rate=0.01, max_depth=4, min_child_weight=18, n_estimators=100, nthread=1, subsample=0.5)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 1)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

print(accuracy_score(testing_target,results))
print(classification_report(testing_target,results))

0.7938144329896907
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       116
           1       0.81      0.64      0.71        78

    accuracy                           0.79       194
   macro avg       0.80      0.77      0.78       194
weighted avg       0.80      0.79      0.79       194



# **Credit Card Dataset from Kaggle**
medium size dataset 

In [54]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: application_record.csv  
  inflating: credit_record.csv       


In [55]:
data = pd.read_csv('/content/application_record.csv')
record = pd.read_csv('/content/credit_record.csv')

In [56]:
# find all users' account open month.
begin_month=pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min))
begin_month=begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'}) 
new_data=pd.merge(data,begin_month,how="left",on="ID") #merge to record data

Preprocessing

In [57]:
record['dep_value'] = None
record['dep_value'][record['STATUS'] =='2']='Yes' 
record['dep_value'][record['STATUS'] =='3']='Yes' 
record['dep_value'][record['STATUS'] =='4']='Yes' 
record['dep_value'][record['STATUS'] =='5']='Yes' 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [58]:
cpunt=record.groupby('ID').count()
cpunt['dep_value'][cpunt['dep_value'] > 0]='Yes' 
cpunt['dep_value'][cpunt['dep_value'] == 0]='No' 
cpunt = cpunt[['dep_value']]
new_data=pd.merge(new_data,cpunt,how='inner',on='ID')
new_data['target']=new_data['dep_value']
new_data.loc[new_data['target']=='Yes','target']=1
new_data.loc[new_data['target']=='No','target']=0

In [59]:
print(cpunt['dep_value'].value_counts())
cpunt['dep_value'].value_counts(normalize=True)

No     45318
Yes      667
Name: dep_value, dtype: int64


No     0.985495
Yes    0.014505
Name: dep_value, dtype: float64

In [60]:
new_data.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Reality',
                         'CNT_CHILDREN':'ChldNo','AMT_INCOME_TOTAL':'inc',
                         'NAME_EDUCATION_TYPE':'edutp','NAME_FAMILY_STATUS':'famtp',
                        'NAME_HOUSING_TYPE':'houtp','FLAG_EMAIL':'email',
                         'NAME_INCOME_TYPE':'inctp','FLAG_WORK_PHONE':'wkphone',
                         'FLAG_PHONE':'phone','CNT_FAM_MEMBERS':'famsize',
                        'OCCUPATION_TYPE':'occyp'
                        },inplace=True)

In [61]:
new_data.dropna()
new_data = new_data.mask(new_data == 'NULL').dropna()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [62]:
new_data.head(3)

,ID,Gender,Car,Reality,ChldNo,inc,inctp,edutp,famtp,houtp,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,wkphone,phone,email,occyp,famsize,begin_month,dep_value,target
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-29.0,No,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-4.0,No,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-26.0,No,0


In [63]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [64]:
new_data['Gender'] = le.fit_transform(new_data['Gender'])
new_data['Car'] = le.fit_transform(new_data['Car'])
new_data['Reality'] = le.fit_transform(new_data['Reality'])
new_data['inctp'] = le.fit_transform(new_data['inctp'])
new_data['edutp'] = le.fit_transform(new_data['edutp'])
new_data['famtp'] = le.fit_transform(new_data['famtp'])
new_data['houtp'] = le.fit_transform(new_data['houtp'])
new_data['occyp'] = le.fit_transform(new_data['occyp'])
new_data['famtp'] = le.fit_transform(new_data['famtp'])

new_data.reset_index(inplace=True)
new_data['target'] = new_data['target'].astype('int')
new_data = new_data.drop(columns=['dep_value','ID','index'])

In [73]:
new_data['target'].value_counts()

0    24712
1      422
Name: target, dtype: int64

In [69]:
X, y = new_data.iloc[:,:-1], new_data.iloc[:, -1]
print(X.shape, y.shape) 
X = np.array(X)
y = np.array(y)

(25134, 18) (25134,)


In [105]:
#over_sampling
from imblearn.over_sampling import SVMSMOTE
from collections import Counter

sm = SVMSMOTE(random_state=42)

print('Original dataset shape %s' % Counter(y))
X_res, y_res = sm.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({0: 24712, 1: 422})
Resampled dataset shape Counter({0: 24712, 1: 24712})


In [106]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

training_features, testing_features, training_target, testing_target = \
            train_test_split(X_res,y_res, random_state=1)

#Randomly selected algorithm with default parameters 
tic()
exported_pipeline = RandomForestClassifier()
exported_pipeline.fit(training_features, training_target)
tac()
results = exported_pipeline.predict(testing_features)
print(accuracy_score(testing_target,results))
print(classification_report(testing_target,results))

Time passed: 0hour:0min:4sec
0.9910974425380382
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6272
           1       0.99      0.99      0.99      6084

    accuracy                           0.99     12356
   macro avg       0.99      0.99      0.99     12356
weighted avg       0.99      0.99      0.99     12356



In [107]:
#### Using TPOT to identify perfect algorithm + parameter

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
model = TPOTClassifier(generations=5, population_size=50,scoring='accuracy', cv=cv,verbosity=2, random_state=1, n_jobs=-1)

tic()
# perform the search
model.fit(X_res, y_res)
# export the best model
model.export('tpot_audit_best_model.py')
tac()



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBClassifier(DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=2, min_samples_leaf=13, min_samples_split=8), learning_rate=0.5, max_depth=4, min_child_weight=13, n_estimators=100, nthread=1, subsample=0.9500000000000001)
Time passed: 0hour:7min:13sec
